In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Reading in the data
cal = pd.read_csv('calendar.csv')
stv = pd.read_csv('sales_train_validation.csv')
ss = pd.read_csv('sample_submission.csv')
sellp = pd.read_csv('sell_prices.csv')

# Downcasting

In this step was performed a downcasting of the dataframes to reduce the amount of storage used by them and also to expidite the operations performed on them.

 - Numerical Columns: Depending on your environment, pandas automatically create int32, int64, float32 or float64 columns for numeric ones. By knowing the min or max value of a column, a subtype can be used to consume less memory. There are the different subtypes:
	- int8 / uint8 : consumes 1 byte of memory, range between -128/127 or 0/255
	- bool : consumes 1 byte, true or false
 	- float16 / int16 / uint16: consumes 2 bytes of memory, range between -32768 and 32767 or 0/65535
	- float32 / int32 / uint32 : consumes 4 bytes of memory, range between -2147483648 and 2147483647
	- float64 / int64 / uint64: consumes 8 bytes of memory

If one of the column has values between 1 and 10 for example, we can reduce the size of that column from 8 bytes per row to 1 byte, which is more than 85% memory saving on that column!

 - Categorical Columns: Pandas stores categorical columns as objects. One of the reason this storage is not optimal is that it creates a list of pointers to the memory address of each value of your column. For columns with low cardinality (the amount of unique values is lower than 50% of the count of these values), this can be optimized by forcing pandas to use a virtual mapping table where all unique values are mapped via an integer instead of a pointer. This can be done using the category datatype.

In [2]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i, t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == object:  # Corrected this line
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

# Apply downcast function to the df_m5s
stv_downcast = downcast(stv)
sellp_downcast = downcast(sellp)
cal_downcast = downcast(cal)

In this step the "sales_train_validation" dataset was melted in order to be able to join its information with oother datasets (such as calendar and sell_prices). 

What the melt function is doing basically is converting the sales dataframe from wide format to a long format. since variables like id, item_id, dept_id, cat_id, store_id and state_id are important to distinguish each product and the store or state they are sold in, they were kept in the new melted dataset.

In [3]:
df = pd.melt(stv_downcast, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

In [4]:
df = pd.merge(df, cal_downcast, on='d', how='left')

In [5]:
df_m5 = pd.merge(df, sellp_downcast, on=['store_id','item_id', 'wm_yr_wk'], how='left')

In [6]:
df_m5

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.980469
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,2.480469
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.980469
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.280273


Due to shortage of memory, the computer used cannot make analysis with such amount of data. To compensate for this the datset was reduced to only have information about the sales of a single item in a store of the California state. 

In [7]:
# Filter data for 'CA' state
df_fd = df_m5[df_m5['dept_id'] == 'FOODS_1']
df_ca = df_fd[df_fd['store_id'] == 'CA_1']

In [8]:
df_ca

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.000000
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,7.878906
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.880859
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.939453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58298703,FOODS_1_215_CA_1_validation,FOODS_1_215,FOODS_1,FOODS,CA_1,CA,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.839844
58298704,FOODS_1_216_CA_1_validation,FOODS_1_216,FOODS_1,FOODS,CA_1,CA,d_1913,1,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,5.281250
58298705,FOODS_1_217_CA_1_validation,FOODS_1_217,FOODS_1,FOODS,CA_1,CA,d_1913,3,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,3.539062
58298706,FOODS_1_218_CA_1_validation,FOODS_1_218,FOODS_1,FOODS,CA_1,CA,d_1913,6,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,0.979980


# Feature Engeneering

Time Series data must be re-framed as a supervised learning dataset before we can start using machine learning algorithms.

There is no concept of input and output features in time series. Instead, we must choose the variable to be predicted and use feature engineering to construct all of the inputs that will be used to make predictions for future time steps.

Lag features are the classical way that time series forecasting problems are transformed into supervised learning problems.

Introduce lags to the the target variable sold. The maximum lag I have introduced is 28 days (4 weeks). 

In [9]:
# Part 4: Lag and Rolling Features
# Lag Features
lags = 28
for lag in range(1, lags + 1):
    df_ca[f'sales_lag_{lag}'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)

C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\2103130892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca[f'sales_lag_{lag}'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\2103130892.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca[f'sales_lag_{lag}'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],

From a mathematical point of view, mean encoding represents a probability of your target variable, conditional on each value of the feature. In a way, it embodies the target variable in its encoded value. The calculated mean encodings on the basis of following logical features were:
 - item
 - state
 - store
 - category
 - department
 - category and department
 - store and item
 - category and item
 - department and item
 - state and store
 - state, store and category
 - store, category and department

In [11]:
# Mean Encoding Features
df_ca['iteam_sold_avg'] = df_ca.groupby('item_id')['sold'].transform('mean').astype(np.float16)
df_ca['state_sold_avg'] = df_ca.groupby('state_id')['sold'].transform('mean').astype(np.float16)
df_ca['store_sold_avg'] = df_ca.groupby('store_id')['sold'].transform('mean').astype(np.float16)
df_ca['cat_sold_avg'] = df_ca.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df_ca['dept_sold_avg'] = df_ca.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
df_ca['cat_dept_sold_avg'] = df_ca.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df_ca['store_item_sold_avg'] = df_ca.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df_ca['cat_item_sold_avg'] = df_ca.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df_ca['dept_item_sold_avg'] = df_ca.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df_ca['state_store_sold_avg'] = df_ca.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df_ca['state_store_cat_sold_avg'] = df_ca.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df_ca['store_cat_dept_sold_avg'] = df_ca.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\449548014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca['iteam_sold_avg'] = df_ca.groupby('item_id')['sold'].transform('mean').astype(np.float16)
C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\449548014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca['state_sold_avg'] = df_ca.groupby('state_id')['sold'].transform('mean').astype(np.float16)
C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\449548014.py

This method is called the rolling window method because the window would be different for every data point.

I'll be calculating weekly rolling avearge of the items sold.

In [10]:
# Rolling Features
df_ca['rolling_mean'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window = 7).mean()).astype(np.float16)
df_ca['rolling_std'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window = 7).std()).astype(np.float16)

C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\3971865725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca['rolling_mean'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window = 7).mean()).astype(np.float16)
C:\Users\Catarina Ferreira\AppData\Local\Temp\ipykernel_8056\3971865725.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ca['rolling_std'] = df_ca.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id

Looking at the new dataframe created after the feature engeneering.

In [12]:
df_ca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413208 entries, 1612 to 58298707
Data columns (total 64 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   id                        413208 non-null  category      
 1   item_id                   413208 non-null  category      
 2   dept_id                   413208 non-null  category      
 3   cat_id                    413208 non-null  category      
 4   store_id                  413208 non-null  category      
 5   state_id                  413208 non-null  category      
 6   d                         413208 non-null  object        
 7   sold                      413208 non-null  int16         
 8   date                      413208 non-null  datetime64[ns]
 9   wm_yr_wk                  413208 non-null  int16         
 10  weekday                   413208 non-null  category      
 11  wday                      413208 non-null  int8          
 1

In [13]:
df_ca_new = df_ca
df_ca_new

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,store_sold_avg,cat_sold_avg,dept_sold_avg,cat_dept_sold_avg,store_item_sold_avg,cat_item_sold_avg,dept_item_sold_avg,state_store_sold_avg,state_store_cat_sold_avg,store_cat_dept_sold_avg
1612,FOODS_1_001_CA_1_validation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1.374023,1.374023,1.374023,1.374023,0.785645,0.785645,0.785645,1.374023,1.374023,1.374023
1613,FOODS_1_002_CA_1_validation,FOODS_1_002,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1.374023,1.374023,1.374023,1.374023,0.477783,0.477783,0.477783,1.374023,1.374023,1.374023
1614,FOODS_1_003_CA_1_validation,FOODS_1_003,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1.374023,1.374023,1.374023,1.374023,0.832031,0.832031,0.832031,1.374023,1.374023,1.374023
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,...,1.374023,1.374023,1.374023,1.374023,8.257812,8.257812,8.257812,1.374023,1.374023,1.374023
1616,FOODS_1_005_CA_1_validation,FOODS_1_005,FOODS_1,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,...,1.374023,1.374023,1.374023,1.374023,1.162109,1.162109,1.162109,1.374023,1.374023,1.374023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58298703,FOODS_1_215_CA_1_validation,FOODS_1_215,FOODS_1,FOODS,CA_1,CA,d_1913,0,2016-04-24,11613,...,1.374023,1.374023,1.374023,1.374023,1.100586,1.100586,1.100586,1.374023,1.374023,1.374023
58298704,FOODS_1_216_CA_1_validation,FOODS_1_216,FOODS_1,FOODS,CA_1,CA,d_1913,1,2016-04-24,11613,...,1.374023,1.374023,1.374023,1.374023,0.776367,0.776367,0.776367,1.374023,1.374023,1.374023
58298705,FOODS_1_217_CA_1_validation,FOODS_1_217,FOODS_1,FOODS,CA_1,CA,d_1913,3,2016-04-24,11613,...,1.374023,1.374023,1.374023,1.374023,2.103516,2.103516,2.103516,1.374023,1.374023,1.374023
58298706,FOODS_1_218_CA_1_validation,FOODS_1_218,FOODS_1,FOODS,CA_1,CA,d_1913,6,2016-04-24,11613,...,1.374023,1.374023,1.374023,1.374023,11.703125,11.703125,11.703125,1.374023,1.374023,1.374023


Save the new dataset in a csv file in order to use it for the prediction models. 

In [15]:
df_ca_new.to_csv('df_ca_new.csv')